In [13]:
import pandas as pd
import glob 
import time
import duckdb

In [14]:
conn = duckdb.connect()

### 01. Data Modelling

In [15]:
# Manually define the order of schema files
schema_files_order = [
    "dim_seller_locations.sql", 
    "dim_product_categories.sql", 
    "dim_payment_methods.sql", 
    "dim_geography.sql", 
    "dim_demographics.sql", 
    "dim_delivery_status.sql", 
    "dim_brands.sql", 
    "dim_customers.sql", 
    "dim_products.sql", 
    "dim_sellers.sql", 
    "sales_fact.sql"
]

schema_directory = "./schema/"

# Execute each SQL file in the correct order
for sql_file_name in schema_files_order:
    sql_file_path = schema_directory + sql_file_name
    with open(sql_file_path, 'r') as file:
        sql_query = file.read()
    conn.execute(sql_query)

In [16]:
# Define the directory where CSV data is stored
csv_directory = "./olap-table/"

# List of CSV file names corresponding to the schema
csv_files = [
    ("seller_locations.csv", "dim_seller_locations"),
    ("product_categories.csv", "dim_product_categories"),
    ("payment_methods.csv", "dim_payment_methods"),
    ("geography.csv", "dim_geography"),
    ("demographics.csv", "dim_demographics"),
    ("delivery_status.csv", "dim_delivery_status"),
    ("brands.csv", "dim_brands"),
    ("customers.csv", "dim_customers"),
    ("products.csv", "dim_products"),
    ("sellers.csv", "dim_sellers"),
    ("sales_fact.csv", "sales_fact"),
]

# Load data into DuckDB from CSV files
for csv_file, table_name in csv_files:
    csv_path = csv_directory + csv_file
    conn.execute(f"COPY {table_name} FROM '{csv_path}' (DELIMITER ',', HEADER TRUE);")

### 2. Configuring the OLAP Cube

- Pre-aggregating the data to mimic the behavior of an OLAP cube.
- Storing these aggregations in tables or materialized views for reusability.

In [ ]:
# Define aggregation queries for OLAP functionality
olap_queries = [
    # Roll-up: Aggregate sales by product category and geography
    {
        "name": "sales_by_category_region",
        "query": """
            CREATE TABLE sales_by_category_region AS
            SELECT
                pc.Category AS product_category,
                geo.Country AS country,
                geo.State AS state,
                SUM(fact.Total_Amount) AS total_sales,
                COUNT(fact.Order_ID) AS transaction_count
            FROM sales_fact fact
            JOIN dim_products prod ON fact.Product_ID = prod.Product_ID
            JOIN dim_product_categories pc ON prod.Category_ID = pc.Category_ID
            JOIN dim_customers cust ON fact.Customer_ID = cust.Customer_ID
            JOIN dim_geography geo ON cust.Geography_ID = geo.Geography_ID
            GROUP BY ROLLUP (pc.Category, geo.Country, geo.State);
        """
    },
    # Drill-down: Aggregate sales by product and customer demographics
    {
        "name": "sales_by_product_customer",
        "query": """
            CREATE TABLE sales_by_product_customer AS
            SELECT
                prod.Product_Name AS product_name,
                demo.Age_Group AS customer_age_group,
                demo.Gender AS customer_gender,
                SUM(fact.Total_Amount) AS total_sales,
                AVG(fact.Total_Amount) AS average_order_value,
                COUNT(fact.Order_ID) AS purchase_frequency
            FROM sales_fact fact
            JOIN dim_products prod ON fact.Product_ID = prod.Product_ID
            JOIN dim_customers cust ON fact.Customer_ID = cust.Customer_ID
            JOIN dim_demographics demo ON cust.Demographics_ID = demo.Demographics_ID
            GROUP BY prod.Product_Name, demo.Age_Group, demo.Gender;
        """
    },
    # Time-based aggregation: Sales by year and month
    {
        "name": "sales_by_time",
        "query": """
            CREATE TABLE sales_by_time AS
            SELECT
                EXTRACT(YEAR FROM fact.Order_Date) AS year,
                EXTRACT(MONTH FROM fact.Order_Date) AS month,
                SUM(fact.Total_Amount) AS total_sales,
                COUNT(fact.Order_ID) AS transaction_count
            FROM sales_fact fact
            GROUP BY CUBE (EXTRACT(YEAR FROM fact.Order_Date), EXTRACT(MONTH FROM fact.Order_Date));
        """
    },
    # # Analyze how delivery status impacts sales.
    # {
    #     "name": "sales_by_delivery_status",
    #     "query": """
    #         CREATE TABLE sales_by_delivery_status AS
    #         SELECT
    #             ds.Delivery_Status AS delivery_status,
    #             SUM(fact.Total_Amount) AS total_sales,
    #             COUNT(fact.Order_ID) AS transaction_count
    #         FROM sales_fact fact
    #         JOIN dim_delivery_status ds ON fact.Delivery_Status_ID = ds.Delivery_Status_ID
    #         GROUP BY ds.Delivery_Status;
    #     """
    # },
    # # Compare sales performance across different payment methods.
    # {
    #     "name": "sales_by_payment_method",
    #     "query": """
    #         CREATE TABLE sales_by_payment_method AS
    #         SELECT
    #             pm.Payment_Method AS payment_method,
    #             SUM(fact.Total_Amount) AS total_sales,
    #             COUNT(fact.Order_ID) AS transaction_count
    #         FROM sales_fact fact
    #         JOIN dim_payment_methods pm ON fact.Payment_Method_ID = pm.Payment_Method_ID
    #         GROUP BY pm.Payment_Method;
    #     """
    # }
]

# Execute OLAP queries to create pre-aggregated tables
for olap in olap_queries:
    conn.execute(olap["query"])
    print(f"Created OLAP table: {olap['name']}")

### 3. Perform OLAP Operations <br/>

#### a) Roll-up (Summarize Data)
Total sales by product category across all regions.

In [19]:
rollup_query = """
SELECT product_category, SUM(total_sales) AS total_sales
FROM sales_by_category_region
GROUP BY product_category;
"""
rollup_result = conn.execute(rollup_query).fetch_df()
print("Roll-up Result:")
print(rollup_result)

Roll-up Result:
  product_category  total_sales
0  Home Appliances  23555751.36
1            Books  23996278.00
2           Beauty  25234977.52
3      Electronics  25157359.96
4        Groceries  24107356.25
5          Fashion  25526498.03


In [30]:
# Step 2: Define OLAP operations
rollup_query = """
    SELECT 
        product_category,
        country,
        SUM(total_sales) AS total_sales,
        SUM(transaction_count) AS total_transactions
    FROM sales_by_category_region
    GROUP BY product_category, country
    ORDER BY total_sales DESC;
"""
rollup_result = conn.execute(rollup_query).fetch_df()
print("Roll-up Result:")
print(rollup_result)

Roll-up Result:
  product_category country  total_sales  total_transactions
0          Fashion   India  25526498.03              8636.0
1           Beauty   India  25234977.52              8576.0
2      Electronics   India  25157359.96              8519.0
3        Groceries   India  24107356.25              8148.0
4            Books   India  23996278.00              8150.0
5  Home Appliances   India  23555751.36              7971.0


#### Drill-down (Detailed Data)
Total sales by individual product for a specific customer age group.

In [27]:
drilldown_query = """
SELECT product_name, total_sales
FROM sales_by_product_customer
WHERE customer_age_group = '26-35';
"""
drilldown_result = conn.execute(drilldown_query).fetch_df()
print("Drill-down Result:")
print(drilldown_result)

Drill-down Result:
       product_name  total_sales
0      Product_5774      6739.58
1      Product_4839       770.20
2      Product_5728      1163.35
3      Product_6007       754.81
4      Product_3800      9726.13
...             ...          ...
10355  Product_3255      7678.18
10356  Product_5493      7673.29
10357  Product_7509      1192.95
10358  Product_3406      3662.27
10359  Product_4482       699.77

[10360 rows x 2 columns]


In [31]:
# Drill-down: Get detailed data
drilldown_query_2 = """
SELECT 
    product_name,
    customer_age_group,
    customer_gender,
    total_sales
FROM sales_by_product_customer
WHERE customer_age_group = '18-25'
ORDER BY total_sales DESC;
"""

drilldown_result_2 = conn.execute(drilldown_query_2).fetch_df()
print("Drill-down Result:")
print(drilldown_result_2)

Drill-down Result:
       product_name customer_age_group customer_gender  total_sales
0      Product_8840              18-25          Female     23151.14
1      Product_9653              18-25            Male     21992.52
2      Product_2231              18-25           Other     20321.79
3      Product_9018              18-25           Other     20097.86
4      Product_7866              18-25            Male     19584.20
...             ...                ...             ...          ...
10593  Product_9013              18-25          Female      -141.92
10594  Product_6522              18-25           Other      -160.89
10595  Product_8488              18-25           Other      -161.11
10596  Product_4706              18-25          Female      -174.19
10597  Product_2279              18-25          Female      -174.83

[10598 rows x 4 columns]


### Slice and Dice (Subset of Data)
Filter sales for a specific region and product category.

In [32]:
# Slice and Dice: View specific subsets of data
slice_dice_query = """
SELECT 
    year,
    month,
    total_sales
FROM sales_by_time
WHERE year = 2023 AND month IN (5, 6, 7)  -- Slice for specific year and months
ORDER BY year, month;
"""

slice_dice_query_result = conn.execute(slice_dice_query).fetch_df()
print("Slide-and-Dice Result:")
print(slice_dice_query_result)

Slide-and-Dice Result:
   year  month  total_sales
0  2023      5  12479284.45
1  2023      6  12456978.49
2  2023      7  12422850.89


#### Pivot (Reorient Data Dimensions)
Pivot total sales by region for each product category.

In [33]:
# Pivot: Reorient data dimensions
pivot_query = """
SELECT 
    product_category,
    SUM(CASE WHEN country = 'USA' THEN total_sales ELSE 0 END) AS sales_usa,
    SUM(CASE WHEN country = 'Canada' THEN total_sales ELSE 0 END) AS sales_canada,
    SUM(CASE WHEN country = 'UK' THEN total_sales ELSE 0 END) AS sales_uk
FROM sales_by_category_region
GROUP BY product_category;
"""
pivot_result = conn.execute(pivot_query).fetch_df()
print("Pivot Result:")
print(pivot_result)

Pivot Result:
  product_category  sales_usa  sales_canada  sales_uk
0  Home Appliances        0.0           0.0       0.0
1            Books        0.0           0.0       0.0
2           Beauty        0.0           0.0       0.0
3      Electronics        0.0           0.0       0.0
4        Groceries        0.0           0.0       0.0
5          Fashion        0.0           0.0       0.0


### 3. Document Queries
Use Python to document all queries programmatically.

In [34]:
documentation = [
    {
        "table": "sales_by_category_region",
        "purpose": "Pre-computed sales totals by product category and region.",
        "query": olap_queries[0]["query"]
    },
    {
        "table": "sales_by_product_customer",
        "purpose": "Pre-computed sales totals by product and customer demographics.",
        "query": olap_queries[1]["query"]
    },
    {
        "table": "sales_by_time",
        "purpose": "Pre-computed sales totals by year and month.",
        "query": olap_queries[2]["query"]
    }
]

# Save documentation to a file
with open("olap_documentation.txt", "w") as doc_file:
    for entry in documentation:
        doc_file.write(f"Table: {entry['table']}\n")
        doc_file.write(f"Purpose: {entry['purpose']}\n")
        doc_file.write(f"Query:\n{entry['query']}\n\n")

In [35]:
conn.close()